# 0. Thêm thư viện

Các thư viện cần thiết, trong đó có `yfinance` để lấy data

In [1]:
import pandas as pd

from copy import deepcopy
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler

import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.regularizers import l1, l2
from keras.optimizers import Adam

import keras_tuner as kt

import tensorflow as tf
from tensorflow.keras import backend as K

import yfinance as yf

from sklearn import tree
import xgboost as xgb

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")


# 1. Lấy Data

hàm lấy data, là cổ phiếu của 50 công ty trên sàn `EURO_STOXX_50` \
Kết quả trả về là 1 dataframe có dạng m dòng, 50 cột với m là time range 

In [ ]:
def EU_Stock_data(start_time,end_time, time_range = 'max'):
    """Lấy dữ liệu giá Close của 50 công ty trên sàn Euro_STOXX 50 vào thời gian cho trước"""

    stock_list = pd.read_html( 'https://en.wikipedia.org/wiki/EURO_STOXX_50')[4]['Ticker'].to_list()

    futures = pd.DataFrame()  

    # xét từng mã
    for symbol in stock_list:
        try:
            df = yf.Ticker(symbol).history(period = time_range, start = start_time, end = end_time)
            df = pd.DataFrame(df['Close']) # lấy giá close
            i = 0
            daily_return = []
            # tinh daily return, = 0 trong ngày đầu tiên 
            for k in df['Close']:
                if i != 0:
                    daily_return.append(float((k-i)/i))
                else:
                    daily_return.append(float(0))
                i = k
            df['Close'] = daily_return
            df.columns = [symbol]
            df.index = df.index.date
            futures = pd.concat([futures,df],axis = 1, join = 'outer').sort_index()
        except:
            continue

    futures['Date'] = pd.to_datetime(futures.index, format='%Y-%m-%d')
    futures.set_index('Date', inplace=True)

    return futures

# 2. Classic TSMOM

Hàm thực hiện tính toán để lấy về giá trị volatility (biến động) của mỗi ngày

In [ ]:
def Volatility_scale(data, ignore_na=False, adjust = True, com = 60, min_periods=0):
    """Scale data using ex ante volatility"""

    # Lưu trữ index, tức thời gian 
    std_index = data.index

    # chứa kết quả
    daily_index = pd.DataFrame(index=std_index)

    # xét từng cổ phiếu
    for oo in data.columns:
        returns = data[oo]  # Lấy ra các return
        returns.dropna(inplace=True)  # xử lý null bằng zero

        # Tính cumulative (cum) return , nhưng ko có thành phần - 1
        ret_index = (1 + returns).cumprod()

        # Tính daily volatility (vol)
        day_vol = returns.ewm(ignore_na=ignore_na,
                              adjust=adjust,
                              com=com,
                              min_periods=min_periods).std(bias=False)
        
        vol = day_vol * np.sqrt(252)  # scale lại theo 252 ngày active trading

        # Join cum return và vol
        ret_index = pd.concat([ret_index, vol], axis=1)
        ret_index.columns = [oo, oo + '_Vol']  # Đặt tên cột cum return là tên cổ phiếu, bên cạnh là vol 

        # Join 
        daily_index = pd.concat([daily_index, ret_index], axis=1)

    return daily_index


Hàm implement chiến lược TSMOM, với logic cụ thể như sau:
Tại ngày t ta so  với ngày t - k về trước, cụ thể ta có thể lấy giá close,
 hoặc cumulative return (nhưng không có thành phần - 1, tức $\text{cum return}_t = \prod_{i = 0}^{t} (1 + r_i)$), 
ở đây xét `cum_return_t` với của k ngày trước
`cum_return_{t-k}`
  - Giả sử `cum_return_t` > `cum_return_{t-k}` tức `sign(cum_return_t - cum_return_{t - k}) = 1` (hàm dấu trả về 1 nếu input > 0)  thì ta có signal = 1, tức đó là tín hiệu để vào lệnh long vào ngày mai 
(ngày t + 1), ngược lại thì signal = -1, là tín hiệu vào short
  -  Sau đó hold trong h -1 ngày tiếp theo (ngày t + 1 vào long đã bắt đầu tính là hold). 
  - Trong các ngày này (tức t + i với i từ 1 đến h), đều có sinh ra Profit and Loss (PnL)  tính theo công thức:\
 ` 0.4/ vol_t * return_{t, t + i}` với `return_{t, t + i}` là return trong giai đoạn t đến t + i, tính tùy vào trường hợp long hay short:
      - nếu long, `return_{t, t + i}` = 1 - `cum_return_t / cum_return_{t + i}`
      - nếu short, `return_{t, t + i}` =  1 - `cum_return_{t + i} / cum_return_t` 
      
    và Leverage, là ` target_vol / vol_t`   (target_vol đang để là 0.4)
 
 Tóm lại, Các kết quả trả về lần lượt là: 
- profit and loss `pnl` 
- `leverage`
- `signal`

In [ ]:
def classic_TSMOM(data, k, h, tolerance = 0,ignore_na = False, adjust = True, com = 60, min_periods = 0):
    signal = pd.DataFrame(index = data.index)

    # gọi hàm Volatility scale
    daily_index = Volatility_scale(data,ignore_na=ignore_na,
                          adjust=adjust,
                          com=com,   
                          min_periods = min_periods)

    company = data.columns

    for oo in company:
        flag_h = 0
        flag_k = k+1
        df = pd.concat([daily_index[oo], daily_index[oo+"_Vol"]], axis=1)
        df = df.dropna(axis = 0, how = 'all')
        df['rolling returns'] = df[oo].pct_change(k) # so sánh thay đổi ở ngày t với k ngày trước đó (tức t - k)
        df['signal'] = 0.
        for x, v in enumerate(df['rolling returns']):
            if flag_h != 0:
                # Bỏ qua giai đoạn hold, tránh bị tính lặp lại
                flag_h = flag_h - 1
                continue
            # Bỏ qua thời gian cty chưa được lên sàn (nêu có)
            if df[oo].isnull().iloc[x] == False:
                # bỏ qua k ngày đầu vì chưa đủ k lookback
                if flag_k != 0:
                    flag_k = flag_k - 1
                    continue
            else: continue
            try:
                if df['rolling returns'].iloc[x-1] < tolerance:
                    for h_period in range(0,h):
                        # rolling return < 0, short rồi giữ trong h ngày, tính pnl, leverage///
                        df['signal'].iloc[x + h_period] = -1
                
                elif df['rolling returns'].iloc[x-1] > tolerance:
                    for h_period in range(0,h):
                        # rolling return > 0, long rồi giữ trong h ngày, tính pnl, leverage///
                        df['signal'].iloc[x + h_period] = 1

            except:pass
            

            # Đặt flag holding là h - 1, để qua vòng for mới bỏ qua ngày hold, tránh bị tính lặp lại
            if df['rolling returns'].iloc[x-1] != tolerance: flag_h = h - 1

        signal = pd.concat([signal, df['signal']], join = 'outer', axis=1)

    signal.columns = data.columns
    
    return signal

# 3. Model

## 3.1. Feature Engineering

In [2]:
def construct_features_single_asset(df,k,h, linear = False, test = False):
    df = df.dropna(how='any',axis=0) 
    df['Cummulative Return'] = (1+ df['Return Daily']).cumprod(axis = 0)
    df['Mean H Return'] = df["Return Daily"].rolling(h+1).apply(lambda x: x.iloc[range(1,h+1)].mean()).shift(-h)
    df['Next H Return'] = df['Cummulative Return'].pct_change(h).shift(-h)
    df['Square Sum Return'] = df["Return Daily"].rolling(h+1).apply(lambda x: x.iloc[range(1,h+1)].pow(2).sum()).shift(-h)

    for temp in range(k,0,-1):
        df["Before " + str(temp) + " Day" ] = df['Cummulative Return'].rolling(h+1).apply(lambda x: x.iloc[h] / x.iloc[k - temp] - 1)

    if linear == True:
        df['Signal'] = [1 if x > 0 else -1 for x in df['Next H Return']]
    
    if test == True:
        df = df.dropna(how='any',axis=0)
        temp = pd.DataFrame(columns= df.columns)
        n = 0
        while True:
            try:
                temp = pd.concat([temp,df.iloc[[n*h],:]], axis = 0)
                n = n+1
            except: break
        df = temp
    else:
        df = df.dropna(how='any',axis=0)

    return df

In [3]:
def feature_engineering(data,k,h,linear = False):
    company = data.columns
    features = []
    for i in range(k,0,-1):
        features.append("Before " + str(i) + " Day")

    X_train = pd.DataFrame(columns=features)
    if linear == False:
        y_train = pd.DataFrame(columns=["Mean H Return","Square Sum Return"])
    elif linear == True:
        y_train = pd.DataFrame(columns=["Signal"])
    for oo in company:
        df = data[[oo]].copy()
        
        df.columns = ["Return Daily"]

        df = construct_features_single_asset(df,k,h,linear = linear)
        
        X_train = pd.concat([X_train,df[features]],axis = 0)
        if linear == False:
            y_train = pd.concat([y_train,df[["Mean H Return","Square Sum Return"]]],axis = 0)
        elif linear == True:
            y_train = pd.concat([y_train,df[["Signal"]]],axis = 0)
    return [X_train,y_train]

## 3.2. Building Model

### Decision Tree

In [ ]:
def train_decision_tree(data, k, h):

    model = tree.DecisionTreeClassifier()

    X_train,y_train = feature_engineering(data,k,h,linear = True)
    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)
    model= model.fit(X_train, y_train)
    
    return model

### XGBoost

In [ ]:
def train_xgboost(data, k, h):

    model = xgb.XGBRegressor(objective="multi:softmax", num_class = 2,random_state=42)

    X_train,y_train = feature_engineering(data,k,h,linear = True)
    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)
    y_train[y_train == -1] = 0
    model= model.fit(X_train, y_train)
    
    return model

### MLP (supervised)

In [42]:
class MLP_supervised(kt.HyperModel):
    def __init__(self, k):
        self.k = k

    def build(self,hp):
        model = Sequential([
            Dropout(0, input_shape=(self.k+1,)),
            Dense(units=hp.Choice(f"units", [5, 10, 20, 40, 80]),activation = 'tanh'),
            Dropout(rate=hp.Choice("dropout", [0.1, 0.2, 0.3, 0.4, 0.5])),
            Dense(1,activation = 'sigmoid'),
        ])

        model.compile(
            optimizer=Adam(
                learning_rate=hp.Choice("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]),
                clipnorm = hp.Choice("max_grad_norm", [1e-4, 1e-3, 1e-2, 0.1, 1.0, 10.0])
            ),
            loss="binary_crossentropy",
        )
        return model
    def fit(self, hp, model, *args, **kwargs):        

        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256,512,1024,2048]),
            **kwargs, epochs = 100, verbose=1
        )


In [45]:
def train_MLP_supervised(data, data_val,k, h):

    X_train,y_train = feature_engineering(data,k,h,linear = True)
    y_train[y_train == -1] = 0

    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)

    X_val,y_val = feature_engineering(data_val,k,h,linear = True)
    y_val[y_val == -1] = 0

    X_val = np.array(X_val, dtype=np.float64)
    y_val = np.array(y_val, dtype=np.float64)

    
    tuner = kt.RandomSearch(
        MLP_supervised(k = k),
        objective="val_loss",
        max_trials=50,
        overwrite=True,
        directory="tuning_dir",
        project_name="tune_MLP_supervised",
    )

    es = EarlyStopping(monitor='val_loss', verbose=1, patience=25)

    checkpoint_filepath = 'Test/Data/checkpoint_MLP_supervised.model.keras'
    model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor = 'val_loss',
    save_best_only=True)    

    tuner.search(X_train, y_train, callbacks = [es],validation_data=(X_val, y_val))

    hypermodel = MLP_supervised(k = k)
    best_hp = tuner.get_best_hyperparameters()[0]
    model = hypermodel.build(best_hp)

    history = hypermodel.fit(best_hp,model,X_train, y_train,callbacks = [model_checkpoint_callback],validation_data = (X_val, y_val))
    
    return model,history

### Lasso (supervised)

In [ ]:
class Lasso_supervised(kt.HyperModel):
    def __init__(self, k):
        self.k = k

    def build(self,hp):
        model = Sequential([
            Dense(1, input_shape = (self.k+1,),kernel_regularizer = l1(hp.Choice("l1_weight", [1e-5,1e-4, 1e-3, 1e-2, 0.1,])),activation='sigmoid')
        ])

        model.compile(
            optimizer=Adam(
                learning_rate=hp.Choice("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]),
                clipnorm = hp.Choice("max_grad_norm", [1e-4, 1e-3, 1e-2, 0.1, 1.0, 10.0])
            ),
            loss="binary_crossentropy",
        )
        return model
    def fit(self, hp, model, *args, **kwargs):
    
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256,512,1024,2048]),
            **kwargs, epochs = 100, verbose=1
        )


In [ ]:
def train_Lasso_supervised(data, data_val,k, h):


    X_train,y_train = feature_engineering(data,k,h,linear = True)
    y_train[y_train == -1] = 0
    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)

    X_val,y_val = feature_engineering(data_val,k,h,linear = True)
    y_val[y_val == -1] = 0

    X_val = np.array(X_val, dtype=np.float64)
    y_val = np.array(y_val, dtype=np.float64)
    tuner = kt.RandomSearch(
        Lasso_supervised(k = k),
        objective="val_loss",
        max_trials=50,
        overwrite=True,
        directory="tuning_dir",
        project_name="tune_Lasso_supervised",
    )

    es = EarlyStopping(monitor='val_loss', verbose=1, patience=25)

    checkpoint_filepath = 'Test/Data/checkpoint_Lasso_supervised.model.keras'
    model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor = 'val_loss',
    save_best_only=True)    


    tuner.search(X_train, y_train, callbacks = [es],validation_data=(X_val, y_val))

    hypermodel = Lasso_supervised(k = k)
    best_hp = tuner.get_best_hyperparameters()[0]
    model = hypermodel.build(best_hp)

    history = hypermodel.fit(best_hp,model,X_train, y_train,callbacks = [model_checkpoint_callback],validation_data = (X_val, y_val))
    
    return model,history

### MLP (Sharpe Loss optimization)

In [4]:
def sharpe_loss(h):
    def calculation(y_target_dummy, y_pred):

        mean = K.reshape(y_target_dummy[:, 0], (-1, 1))
        square_sum =  K.reshape(y_target_dummy[:, 1], (-1, 1))

        sum_pofolio = mean * h * y_pred
        mean_pofolio = K.mean(mean * h * y_pred) / h

        std_pofolio = tf.math.sqrt(K.mean(square_sum * y_pred **2 
                                          - 2 * sum_pofolio * mean_pofolio 
                                          + (mean_pofolio ** 2) * h)/h)

    
        return  - (mean_pofolio / std_pofolio) *np.sqrt(252)
    
    return calculation

In [8]:
class MLP_SharpeLoss(kt.HyperModel):
    def __init__(self, k,h):
        self.k = k
        self.h = h

    def build(self,hp):
        model = Sequential([
            Dropout(0, input_shape=(self.k+1,)),
            Dense(units=hp.Choice(f"units", [5, 10, 20, 40, 80]),activation = 'tanh'),
            Dropout(rate=hp.Choice("dropout", [0.1, 0.2, 0.3, 0.4, 0.5])),
            Dense(1,activation = 'tanh'),
        ])

        model.compile(
            optimizer=Adam(
                learning_rate=hp.Choice("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]),
                clipnorm = hp.Choice("max_grad_norm", [1e-4, 1e-3, 1e-2, 0.1, 1.0, 10.0])
            ),
            loss= sharpe_loss(h = self.h)
        )
        return model
    def fit(self, hp, model, *args, **kwargs):

        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256,512,1024,2048]),
            **kwargs, epochs = 100, verbose=1
        )


In [16]:
def train_MLP_sharpeLoss(data, data_val, k, h):

    X_train,y_train = feature_engineering(data,k,h,linear = False)

    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)

    X_val,y_val = feature_engineering(data_val,k,h,linear = False)

    X_val = np.array(X_val, dtype=np.float64)
    y_val = np.array(y_val, dtype=np.float64)
    tuner = kt.RandomSearch(
        MLP_SharpeLoss(k = k,h=h),
        objective="val_loss",
        max_trials=50,
        overwrite=True,
        directory="tuning_dir",
        project_name="tune_MLP_sharpeLoss",
    )

    es = EarlyStopping(monitor='val_loss', verbose=1, patience=25)

    checkpoint_filepath = 'Test/Data/checkpoint_MLP_sharpeLoss.model.keras'
    model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor = 'val_loss',
    save_best_only=True)    


    tuner.search(X_train, y_train, callbacks = [es], validation_data=(X_val, y_val))

    hypermodel = MLP_SharpeLoss(k = k,h= h)
    best_hp = tuner.get_best_hyperparameters()[0]
    model = hypermodel.build(best_hp)

    history = hypermodel.fit(best_hp,model,X_train, y_train,callbacks = [model_checkpoint_callback],validation_data = (X_val, y_val))
    return model,history

### Lasso (Sharpe Loss optimization)

In [20]:
class Lasso_SharpeLoss(kt.HyperModel):
    def __init__(self, k,h):
        self.k = k
        self.h = h

    def build(self,hp):
        model = Sequential([
            Dense(1, input_shape = (self.k+1,),kernel_regularizer = l1(hp.Choice("l1_weight", [1e-5,1e-4, 1e-3, 1e-2, 0.1,])),activation='sigmoid')
        ])

        model.compile(
            optimizer=Adam(
                learning_rate=hp.Choice("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]),
                clipnorm = hp.Choice("max_grad_norm", [1e-4, 1e-3, 1e-2, 0.1, 1.0, 10.0])
            ),
            loss= sharpe_loss(h = self.h)
        )
        return model
    def fit(self, hp, model, *args, **kwargs):

        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256,512,1024,2048]),
            **kwargs, epochs = 100, verbose=1
        )


In [27]:
def train_Lasso_sharpeLoss(data, data_val,k, h):

    X_train,y_train = feature_engineering(data,k,h,linear = False)

    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)

    X_val,y_val = feature_engineering(data_val,k,h,linear = False)
    
    X_val = np.array(X_val, dtype=np.float64)
    y_val = np.array(y_val, dtype=np.float64)
    tuner = kt.RandomSearch(
        Lasso_SharpeLoss(k = k,h=h),
        objective="val_loss",
        max_trials=50,
        overwrite=True,
        directory="tuning_dir",
        project_name="tune_Lasso_sharpeLoss",
    )

    es = EarlyStopping(monitor='val_loss', verbose=1, patience=25)

    checkpoint_filepath = 'Test/Data/checkpoint_Lasso_sharpeLoss.model.keras'
    model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor = 'val_loss',
    save_best_only=True)    


    tuner.search(X_train, y_train, callbacks = [es], validation_data=(X_val, y_val))

    hypermodel = Lasso_SharpeLoss(k = k,h= h)
    best_hp = tuner.get_best_hyperparameters()[0]
    model = hypermodel.build(best_hp)

    history = hypermodel.fit(best_hp,model,X_train, y_train,callbacks = [model_checkpoint_callback], validation_data = (X_val, y_val))
    return model,history

## 3.3. Test model

In [ ]:
def test_model_TSMOM(data, model,k,h,linear = False):

    company = data.columns

    signal = pd.DataFrame(index = data.index)

    features = []
    for i in range(k,0,-1):
        features.append("Before " + str(i) + " Day")
    features.append("Return Daily")

    for oo in company:
        df = data[[oo]].copy()
        
        df.columns = ["Return Daily"]
        df = construct_features_single_asset(df,k,h,linear = linear,test= True)
        time_index = data[oo].dropna(how = 'any').index
        company_signal = pd.DataFrame(index = time_index, columns = [oo])
        
        X_test = df[features]
        if X_test.shape[0] == 0: continue
        try:
            if model.loss ==  'binary_crossentropy':
                X_test['prediction'] = np.sign(model.predict(X_test) - 0.5)
            else:
                X_test['prediction'] = np.sign(model.predict(X_test))
        except:
            X_test['prediction'] = np.sign(model.predict(X_test))
            X_test['prediction'][X_test['prediction'] == 0] = -1
        for x,v in enumerate(X_test.index):
            company_signal.loc[v,oo] = X_test.loc[v,'prediction']
        
        company_signal = company_signal.ffill()
        company_signal = company_signal.fillna(0)

        signal = pd.concat([signal,company_signal], axis = 1, join = 'outer')


    return signal

# 4. Backtest

In [ ]:
def backtest(data,signal,k,h,  vol_flag = 1, target_vol = 0.2, ignore_na = False, adjust = True, com = 60, min_periods = 0):
    
    pnl = pd.DataFrame(index=data.index)
    leverage = pd.DataFrame(index = data.index)

    # gọi hàm Volatility scale
    daily_index = Volatility_scale(data,ignore_na=ignore_na,
                          adjust=adjust,
                          com=com,   
                          min_periods = min_periods)

    company = signal.columns

    # Volatility settings
    vol_flag = vol_flag    # Set flag to 1 for vol targeting
    if vol_flag == 1:
        target_vol = target_vol 
    else:
        target_vol = 'no target vol'
    

    for oo in company:
        flag_h = 0
        flag_k = k+1
        df = pd.concat([daily_index[oo], daily_index[oo+"_Vol"]], axis=1)
        df = df.dropna(axis = 0, how = 'all')

        company_signal = signal[oo].dropna(axis = 0, how = 'all')
        df['pnl'] = 0. 
        df['leverage'] = 0.
        for x, v in enumerate(df['pnl']):
            if flag_h != 0:
                # Bỏ qua giai đoạn hold, tránh bị tính lặp lại
                flag_h = flag_h - 1
                continue
            # Bỏ qua thời gian cty chưa được lên sàn (nêu có)
            if df[oo].isnull().iloc[x] == False:
                # bỏ qua k ngày đầu vì chưa đủ k lookback
                if flag_k != 0:
                    flag_k = flag_k - 1
                    continue
            else: continue
            try:
                if company_signal.iloc[x] == -1:
                    for h_period in range(0,h):
                        if vol_flag == 1:
                            df['pnl'].iloc[x + h_period] = (1 - df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period]) * \
                                target_vol / df[oo+"_Vol"].iloc[x -1] 
                            df['leverage'].iloc[x + h_period] = target_vol / df[oo+"_Vol"].iloc[x -1]
                        else:
                            df['pnl'].iloc[x + h_period] = (1 - df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period])
                            df['leverage'].iloc[x+h_period] = 1
                elif company_signal.iloc[x] == 1:
                    for h_period in range(0,h):
                        if vol_flag == 1:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period] - 1) * \
                                    target_vol / df[oo+"_Vol"].iloc[x - 1]
                            df['leverage'].iloc[x+h_period] = target_vol / df[oo+"_Vol"].iloc[x -1]
                        else:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period] - 1)
                            df['leverage'].iloc[x+h_period] = 1
            except:pass
            
            if signal[oo].iloc[x] == 1 or signal[oo].iloc[x] == -1 : flag_h = h - 1


        leverage = pd.concat([leverage, df['leverage']], join = 'outer',axis = 1)
        pnl = pd.concat([pnl, df['pnl']], join = 'outer',axis=1)

    pnl.columns = signal.columns
    leverage.columns = signal.columns

    return [pnl,leverage]

Cuối cùng, ta lấy mean của 50 cổ phiếu để có `PnL` đại diện 

In [ ]:
def strategy_daily_return(pnl):
    
    return pnl.mean(axis=1)

# 5. Example Code

## 5.1. Lấy Data

In [ ]:
# # Thời gian input theo dạng yyyy-mm-dd; với ví dụ ở dưới 

# start_time = '2004-12-31'
# end_time = '2010-01-01' 

# df = EU_Stock_data(start_time,end_time)


## 5.2. Code demo Classic TSMOM

In [ ]:
daily_return = EU_Stock_data()
daily_index = Volatility_scale(daily_return)

# print ra result là pnl, leverage, signal của hàm backtest_strategy(), với k = 3, h = 3, target volatility = 0.4
LOOKBACK = 3
HOLDING = 3
TARGET_VOL = 0.4

signal = classic_TSMOM(daily_return,LOOKBACK,HOLDING)
[pnl,leverage] = backtest(daily_return,signal,LOOKBACK,HOLDING, target_vol= TARGET_VOL)

print(f'pnl với k = {LOOKBACK} , h = {HOLDING}, target volatility = {TARGET_VOL}:')
pnl

print(f'leverage với k = {LOOKBACK} , h = {HOLDING}, target volatility = {TARGET_VOL}:')
leverage

print(f'signal với k = {LOOKBACK} , h = {HOLDING}, target volatility = {TARGET_VOL}:')
signal

## 5.3 Code so sánh các cặp k,h khi sử dụng classic TSMOM

In [ ]:
data = pd.read_csv('data.csv' , index_col= 'Date')

for k in [1,3,5,10,15,20,40]:
    for h in [1,3,5,10,15,20,40]:
        # print([k,h]) # Kiểm tra tiến độ
        signal = classic_TSMOM(data,k,h)
        result = backtest(data,signal,k,h)
        # result[0].to_csv("pnl (k = " + str(0 if k < 10 else "") + str(k) + ", h = " + str(0 if h < 10 else "") + str(h) + ").csv")
        # result[1].to_csv("leverage (k = " + str(0 if k < 10 else "") + str(k) + ", h = " + str(0 if h < 10 else "")  + str(h) + ").csv")
        # result[2].to_csv("signal (k = " + str(0 if k < 10 else "") + str(k) + ", h = " + str(0 if h < 10 else "")  + str(h) + ").csv")
        temp = strategy_daily_return(result[0])
        try:
            temp2 = temp.to_list()
            temp2.insert(0,h)
            temp2.insert(0,k)
            stats.loc[len(stats.index)] = temp2
        except:
            index = temp.index.to_list()
            index.insert(0,'h')
            index.insert(0,'k')
            stats = pd.DataFrame(columns = index)
            temp2 = temp.to_list()
            temp2.insert(0,h)
            temp2.insert(0,k)
            stats.loc[len(stats.index)] = temp2
        del result

stats.to_csv("k_h_Comparing.csv")

## 5.4. Code demo thử model

### Train theo tỉ lệ Data (9 năm Train; 1 năm Validation và 5 năm Backtest từ 15 năm)

In [ ]:
start_time = '2009-12-31'
end_time = '2024-12-22'

data = EU_Stock_data(start_time = start_time, end_time=end_time)

k = 40
h = 40

model_name = ['classic_TSMOM','train_decision_tree','train_xgboost','train_MLP_supervised','train_Lasso_supervised','train_MLP_sharpeLoss','train_Lasso_sharpeLoss']

for model in model_name:
    func = globals()[model]
    if model == 'classic_TSMOM':
        test_data = data[data.index > datetime(pd.to_datetime(start_time).year + 10,12,31)]

        signal = func(test_data,k,h)
        # signal.to_csv("signal_" + str(model) + ".csv")
        pnl = strategy_daily_return(backtest(test_data,signal,k,h)[0])

    elif model in ['train_decision_tree','train_xgboost']:
        train_data = data[data.index <= datetime(pd.to_datetime(start_time).year + 10,12,31)]
        test_data = data[data.index > datetime(pd.to_datetime(start_time).year + 10,12,31)]

        temp_model = func(train_data,k,h)
        signal = test_model_TSMOM(test_data,temp_model,k,h)
        # signal.to_csv("signal_" + str(model) + ".csv")
        pnl = strategy_daily_return(backtest(test_data,signal,k,h)[0])

    else:
        train_data = data[data.index <= datetime(pd.to_datetime(start_time).year + 9,12,31)]
        val_data = data[data.index > datetime(pd.to_datetime(start_time).year + 9,12,31) and data.index <= datetime(pd.to_datetime(start_time).year + 10,12,31)]
        test_data = data[data.index > datetime(pd.to_datetime(start_time).year + 10,12,31)]
        temp_model = func(train_data,val_data,k,h)[0]
        signal = test_model_TSMOM(test_data,temp_model,k,h)
        # signal_1.to_csv("signal_" + str(model) + ".csv")
        pnl = strategy_daily_return(backtest(test_data,signal,k,h)[0])

    try:
        if len(stats.columns) > 0:
            temp = pnl
            temp.insert(0,model)
            stats.loc[len(stats.index)] = temp
    except:
        index = pnl.index.to_list()
        index.insert(0,'Model')
        stats = pd.DataFrame(columns = index)
        temp = pnl
        temp.insert(0,model)
        stats.loc[len(stats.index)] = temp


stats.to_csv("result.csv")

### Train theo Rolling Window